In [ ]:
# @title Forest Fires

## IMPORTING LIBRARIES

# We'll import the typing library from Python to add type hints to our code.
from typing import List, Optional, Tuple

# Matplotlib is a useful library of visualization and plotting tools.
import matplotlib as mpl
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

# These libraries will allow us to make the notebook interactive.
from base64 import b64encode
import matplotlib.animation as animation

# NumPy is a popular library for mathematical functions and efficient arrays.
import numpy as np
from IPython.display import HTML, display

# We'll import the convolution operator from the SciPy library.
from scipy.signal import convolve2d

# We'll use the tqdm library to generate some progress bar for our loops.
from tqdm import trange


def visualize_forest_fire(
    name: str,
    grid_size: int,
    num_iterations: int,
    prob_grow: float,
    prob_fire: float,
    interval: int,
    live_lim: int,
    burn_lim: int,
    seed: Optional[int],
) -> None:
    """Visualizes a forest-fire model. The function also saves the animation to an MP4 file.

    Args:
        name: The name of the visualization. The visualization will be saved to an MP4 with this name.
        grid_size: The size of the grid for forest-fire model.
        num_iterations: The number of iterations to simulate the model for.
        prob_grow: The probability that an empty node will grow a tree.
        prob_fire: The probability that an inactive tree will catch on fire (e.g., by lightning).
        interval: The number of iterations to skip by when animating.
        live_lim: The y-axis limit to use for the live trees time series plot.
        burn_lim: The y-axis limit to use for the burning trees time series plot.
        seed: The seed for the random number generator.
    """
    # Initialize the random number generator.
    rng = np.random.default_rng(seed)

    # Initialize the grid to all zeros.
    grids = np.zeros((num_iterations, grid_size, grid_size), dtype="int")

    # Run the forest-fire simulation.
    for step in range(1, num_iterations):
        grids[step] = apply_forest_fire_rule(grids[step - 1], prob_grow, prob_fire, rng)

    # Initialize the figure.
    fig = plt.figure(figsize=(9, 9), layout="tight")
    fig.patch.set_facecolor("none")
    gs = gridspec.GridSpec(3, 1)
    ax1 = fig.add_subplot(gs[:2, :])
    ax2 = fig.add_subplot(gs[2, :])

    # Set up the initial plot.
    grid = grids[0]
    Y, X = np.mgrid[: grid.shape[0], : grid.shape[1]]
    X = X.flatten()
    Y = Y.flatten()
    grid = grid.flatten()

    # Define the colourmap for the plot.
    cmap = mpl.colors.ListedColormap(["#FFFFFF", "#5790FC", "#E42536"])
    norm = mpl.colors.Normalize(vmin=0, vmax=2)

    # Create the scatter plot. This plot will be updated repeatedly by the update function.
    scat = ax1.scatter(X, Y, c=grid, cmap=cmap, norm=norm)

    # Set up the plot axes and text.
    ax1.set_aspect("equal", "box")
    ax1.invert_yaxis()
    ax1.set_xticklabels([])
    ax1.set_yticklabels([])
    ax1.xaxis.set_ticks_position("none")
    ax1.yaxis.set_ticks_position("none")
    ax1.patch.set_facecolor("none")
    ax1.set_title("Forest Fire Simulation", fontsize=16)

    # Calculate some measures across the simulation.
    n_inactive = np.sum(grids == 1, axis=(1, 2))
    n_active = np.sum(grids == 2, axis=(1, 2))

    # Plot the number of live trees over time.
    (line1,) = ax2.plot([], [], color="#5790FC")
    ax2.set_xlim([0, len(n_inactive)])
    ax2.set_xlabel("Iteration")
    ax2.xaxis.set_minor_locator(ticker.AutoMinorLocator())
    ax2.set_ylim([0, live_lim])
    ax2.set_ylabel("Live Trees", color="#5790FC")
    ax2.yaxis.set_minor_locator(ticker.AutoMinorLocator())
    ax2.tick_params(axis="y", which="both", colors="#5790FC")
    ax2.grid(False)

    # Plot the number of burning trees over time on the same plot.
    ax3 = ax2.twinx()
    (line2,) = ax3.plot([], [], color="#E42536")
    ax3.set_ylim([0, burn_lim])
    ax3.set_ylabel("Burning Trees", color="#E42536")
    ax3.tick_params(axis="y", colors="#E42536")
    ax3.yaxis.set_minor_locator(ticker.AutoMinorLocator())
    ax3.tick_params(axis="y", which="both", colors="#E42536")
    ax3.grid(False)

    # Final formatting changes.
    ax2.patch.set_facecolor("none")
    ax3.patch.set_facecolor("none")
    ax3.spines["top"].set_color("black")
    ax3.spines["bottom"].set_color("black")
    ax3.spines["left"].set_color("#5790FC")
    ax3.spines["right"].set_color("#E42536")

    def init():
        scat.set_array(grid)
        line1.set_data([], [])
        line2.set_data([], [])
        return scat, line1, line2

    def update(frame):
        index = frame * interval

        grid = grids[index].flatten()
        scat.set_array(grid)

        line1.set_data(range(index), n_inactive[:index])
        line2.set_data(range(index), n_active[:index])

        return scat, line1, line2

    anim = animation.FuncAnimation(
        fig,
        update,
        frames=num_iterations // interval,
        init_func=init,
        interval=100,
        repeat=False,
        blit=True,
    )

    anim.save(f"{name}.mp4", writer="ffmpeg")

    plt.close(fig)

    # Open the MP4 file and display it in this cell.
    mp4 = open(f"{name}.mp4", "rb").read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    display(HTML(f"""<video src={data_url} controls/>"""))

In a previous notebook, we explored iterated growth with cellular automata. We saw that the cellular automata could produce complex and intricate patterns even when each cell or node had only two states and when using simple rules. In this chapter, we'll extend the use of cellular automata to model forest fires. Again, we'll use Python to produce some neat visualizations. The code and the explanations come from Paul Charbonneau's book _Natural Complexity_; see references below.

For our forest-fire model, consider a 2-D 8-neighbour Cartesian grid where each node can be in one of three states: "empty", "inactive", or "active". Think of each node as a single tree in our simulated forest. Initially, all nodes are in an empty state. As the simulation progresses, the state of each node can change based on a set of rules. Previously, rules were **always** applied to each cell or node; we call these _deterministic_ rules. Now, we'll introduce rules which are applied with a probability _p_; these are called _stochastic_ rules. Our forest-fire model uses four simple rules:

1. An empty node can be set to inactive with probability `prob_grow` (think of this as a tree growing).
2. An inactive node can become active with probability `prob_fire` (this is like a tree catching on fire).
3. An inactive node will always be set to active if any of its neighbouring nodes were active in the previous iteration (this represents fire spreading from tree to tree).
4. An active node will always become empty in the next iteration (this is like a tree burning down completely).

So, as you can see, this model is inspired by how a forest fire spreads. The occupied (inactive or active) nodes represent trees, and the active nodes, in particular, represent burning trees. A tree can catch on fire randomly (like from lightning) or from a neighbouring tree already on fire. Once a tree is on fire, it'll burn down and become an empty node in the next round. This model uses two stochastic rules and two deterministic rules. This model also allows for a _fire front_, which is a big wave of burning trees, to develop when one tree catches on fire and then sets all its neighbours on fire, and then they set their neighbours on fire, and so on. This can create a sort of "avalanche" of burning trees, which is fascinating to watch. But, as we'll see, this isn't the only surprising thing that can happen in this model.

## Basic Implementation

For starters, we'll borrow some code from the previous notebook to get a node's neighbours in a 2-D Cartesian grid. Note that we've modified the code slightly always to return eight neighbours since that's the only case we're considering for our forest-fire model.


In [ ]:
def get_2d_neighbors(x: int, y: int, x_max: int, y_max: int) -> List[Tuple[int, int]]:
    """Calculate and return the valid neighbours of a node at position (x, y).

    Args:
        x: The x-coordinate of the node.
        y: The y-coordinate of the node.
        x_max: The maximum x-coordinate value of the grid.
        y_max: The maximum y-coordinate value of the grid.

    Returns:
        A list of the (x, y) coordinates for each neighbour.
    """
    neighbors = [
        (x + 1, y),      # right neighbour
        (x, y - 1),      # top neighbour
        (x - 1, y),      # left neighbour
        (x, y + 1),      # bottom neighbour
        (x + 1, y - 1),  # top-right neighbour
        (x - 1, y - 1),  # top-left neighbour
        (x - 1, y + 1),  # bottom-left neighbour
        (x + 1, y + 1),  # bottom-right neighbour
    ]

    # Remove any invalid neighbours, such as negative indices.
    neighbors = [(i, j) for (i, j) in neighbors if 0 <= i < x_max and 0 <= j < y_max]

    # Return the list of neighbours.
    return neighbors

Next, we write a function which will apply our rule to each tree in the grid. This function will look at the node's current state (empty, inactive, or active) and return what the next state of that node should be. The function takes six arguments:

1. `row`: The row index of the tree in the grid.
2. `column`: The column index of the tree in the grid.
3. `grid`: The grid of trees. As usual, this will be a 2-D NumPy array.
4. `prob_grow`: The probability that an empty node will grow a tree. This is used in the first rule and should be a number between 0 and 1.
5. `prob_fire`: The probability that an inactive tree will catch on fire. This is used in the second rule and should also be a number between 0 and 1.
6. `rng`: The is a special NumPy object that will let us roll random numbers. We pass this along to the function so that our randomness can be "seeded", which means that while still being random, when we run the notebook and when you run the notebook, we should get the same results. This way, we can talk about the results.

Our function comprises an `if`-block that checks the node's current state in the grid. We represent the state as an integer. 0 represents an empty node, 1 represents an inactive node, and 2 represents an active node. The call to `rng.uniform()` generates a random number between 0 and 1. We take action if this number ends up lower than a probability `p`. Does that make sense? Consider `p = 1`, which means a certain event (an event which always occurs). Regardless of what `rng.uniform()` is, it will be less than 1. So the code will always run. (You may wonder whether we should use `<` or `<=` here. The answer is no because `rng.uniform()` can return 0, but not 1. Therefore, for `p = 0`, if we used `<=`, there is a chance that it resolves to `True`, which should never happen.)

Note the syntax of `any(grid[nrow, ncol] == 2 for nrow, ncol in tree_neighbors)`. This is basically a `for`-loop written in one line. Here, we are looping over each index pair `(nrow, ncol)` in the list `tree_neighbors`. We then check whether `grid[nrow, ncol]` equals `2`, representing an active node. If that condition is met for any of the neighbours, the `any()` function will return `True`.

In [ ]:
def apply_forest_fire_rule(
    row: int,
    col: int,
    grid: np.ndarray,
    prob_grow: float,
    prob_fire: float,
    rng: np.random.Generator,
) -> int:
    """Simulate one iteration of the forest-fire model on the tree at position (x, y).

    Args:
        row: The row index of the tree.
        col: The column index of the tree.
        grid: The 2-D Cartesian grid of trees.
        prob_grow: The probability that an empty node will grow a tree.
        prob_fire: The probability that an inactive tree will catch on fire (e.g., by lightning).
        rng: The random number generator.

    Returns:
        The new state of the tree at position (x, y).
    """
    # Get the current state of the tree.
    # 0 = empty (no tree)
    # 1 = inactive (not on fire)
    # 2 = active (on fire)
    tree_state = grid[row, col]

    # Get the indices of the neighbouring trees.
    tree_neighbors = get_2d_neighbors(row, col, grid.shape[0], grid.shape[1])

    # Rule 1: an empty node can be set to inactive with probability `prob_grow` (think of this as a tree growing).
    if tree_state == 0:
        if rng.uniform() < prob_grow:
            return 1

    # Rule 2: an inactive node can become active with probability `prob_fire` (this is like a tree catching on fire).
    if tree_state == 1:
        if rng.uniform() < prob_fire:
            return 2

    # Rule 3: an inactive node will always be set to active if any of its neighbouring nodes were active in the previous
    # iteration (this represents fire spreading from tree to tree).
    if tree_state == 1 and any(grid[nrow, ncol] == 2 for nrow, ncol in tree_neighbors):
        return 2

    # Rule 4: an active node will always become empty in the next iteration (this is like a tree burning down completely).
    if tree_state == 2:
        return 0

    # Otherwise, the state of the node doesn't change.
    return tree_state

In the following code cell, we define some parameters of our forest-fire model, such as the number of iterations and the probabilities our model will use. We will wait to visualize the output. For now, we'll use a progress bar to time how long it takes for our code to simulate 500 iterations on a 100x100 grid.

In [ ]:
# Define the size of the grid.
grid_size = 100

# Define the number of iterations we want the forest-fire simulation to run.
n_iter = 1000

# Define the probability that a tree grows in an empty node.
prob_grow = 0.001

# Define the probability that an inactive node spontaneously becomes active.
prob_fire = 0.00001

# Initialize the random number generator.
rng = np.random.default_rng(123)

# Initialize the grid to all zeros.
grids = np.zeros((n_iter, grid_size, grid_size), dtype="int")

# Run the simulation
for step in trange(1, n_iter):

    # Loop through each node in the grid
    for row in range(grids.shape[1]):
        for col in range(grids.shape[2]):

            # Call our helper function to determine the state of this node based on the grid's previous state
            grids[step, row, col] = apply_forest_fire_rule(row, col, grids[step - 1], prob_grow, prob_fire, rng)

The code runs and works as expected. However, it could be more efficient, as it takes about two minutes to run 1000 iterations (this number might be different for you). We want to simulate the forest-fire model for much longer. However, the time it takes to run the simulation will keep adding up. In Charbonneau's book chapter, he uses 51000 iterations, which would take over an hour to simulate. Can we make it more efficient?

## A More Efficient Implementation

One way of doing this is by using _masks_. Think of a mask as a filter over the nodes of the grids. The mask filters any nodes that don't satisfy a condition. What this looks like in code is a grid of booleans (True/False values) with the same shape as our model's grid. The mask has the value `True` when the node at that same position satisfies the condition; otherwise, it has the value `False`. The NumPy library we're using makes it easy and efficient to create and apply these masks. So, let's transform our function `apply_forest_fire_rule()` to use masks instead and see what speedups we get.

The first change is that, by using masks, we are no longer looking at nodes individually. Instead, our masks will filter it to only nodes that we want to consider. We can then apply a transformation (such as updating the node's state) to the entire grid in one go. This is significantly more efficient than setting the value one at a time. Because of this, rather than passing the `(row, col)` index of each node and calling it within a nested loop, we can just pass the entire grid to the function. The function will return a new grid with all the nodes updated to their next state.

We'll start with the easiest rule, which is the fourth rule. This rule states that an active node will always become empty in the next iteration. In other words, we must first create a mask of only the active nodes in the grid. We then use this mask to set every node to the empty state. We only need to write `grid == 2` to do the first step. This is some syntax that the NumPy library allows us to do. Recall that `grid` is the 2-D array representing the model's current state. The code creates a mask that checks the value in each node and returns `True` only if that value is `2` (which represents an active node). To use this mask, we use the same indexing notation of square brackets. Rather than passing the row and column indices, we can pass it the mask like so: `grid[grid == 2]`. This returns only the nodes that satisfy the masking condition. We can then update the value of these nodes by setting them to `0` (for an empty node). Note that in the code, we don't want to modify the model's current state but instead create a new grid representing the next state. For this reason, we write `new_grid[grid == 2] = 0`, where `new_grid` is the grid representing the next state.

Rules 1 and 2 are pretty similar. Both rules only consider nodes of a particular state and, with a certain probability, will change their values. Adding in this probability part is quite simple. We need to modify our mask by applying another mask. Before, we rolled a random number between 0 and 1 for each node. Now, we instead roll a number for _every_ node in the grid. If this number is less than our probability value, we set the value to `True`, indicating that we want to change that node. We can combine our two masks using the NumPy's bit-wise AND operator, `&`. So, for rule one, which only considers empty nodes, we first create a mask for empty nodes, `grid == 0`. We then create a grid of random numbers between 0 and 1 with the same shape as our model's grid by writing `rng.uniform(size=grid.shape)`. We then set only the nodes less than the probability that a tree grows to `True` while we set the others to `False`: `rng.uniform(size=grid.shape) < prob_grow`. Finally, combining these two masks gives us our final mask, which we then use to set the nodes to inactive. Rule 2 follows a similar approach.

The most complex rule to implement is rule 3, and this is because we need to consider the neighbours of that node, not just the node itself. However, we can approach it similarly to the previous two rules. First, we create a mask for inactive nodes. Then, we make a mask of only nodes with at least one active neighbour. Finally, we'll combine the two masks and update the correct nodes to the active state. The first step is as before; we can reuse that same mask, `inactive_nodes`, here. However, how do we create the second mask efficiently? There are a few ways to approach this. The method we use involves an operation called a _convolution_. First, we create a mask of _active_ nodes. Then, we take a 3x3 "kernel" (also called a "filter") and apply it to each node in the grid. This kernel counts the number of active nodes in that 3x3 grid around the node and places that value into a new grid. At the end of this operation, we'll have a new grid that contains the number of active neighbours to each node. Finally, to create the mask, we only set the nodes with a value greater than `0` to `True` and the rest to `False`. Convolutions are handy operations on grids (or matrices) and are important in machine learning. They are the integral component of so-called _convolutional neural networks_, which are networks that form the backbone of many image processing and recognition algorithms.

In [ ]:
def get_active_neighbors_mask(grid: np.ndarray) -> np.ndarray:
    """Returns a mask over the input grid, masking only nodes which have at least one active neighbour.

    Args:
        grid: The 2-D Cartesian grid of trees.

    Returns:
        A mask for only nodes with at least one active neighbour.
    """
    # Get a map of the active nodes in the grid.
    active_nodes = grid == 2

    # Define a 3x3 kernel representing the neighbours of the center node.
    neighbor_kernel = np.array(
        [
            [1, 1, 1],
            [1, 0, 1],
            [1, 1, 1],
        ]
    )

    # Apply the convolution across the entire grid.
    #
    # Note the "same" convolution, indicating that the grid is first padded with 0s to ensure that the output grid is
    # the same size as the input grid.
    #
    # `active_neighbors` is thus a grid (with the same shape as `grid`) where each element represents the total number
    # of active neighbours to that node.
    active_neighbors = convolve2d(active_nodes, neighbor_kernel, mode="same")

    # Return a mask that is `True` when there is at least one active neighbour, `False` otherwise.
    return active_neighbors > 0


def apply_forest_fire_rule(
    grid: np.ndarray,
    prob_grow: float,
    prob_fire: float,
    rng: np.random.Generator,
) -> np.ndarray:
    """Simulate one iteration of the forest-fire model on the entire grid.

    Args:
        grid: The 2-D Cartesian grid of trees.
        prob_grow: The probability that an empty node will grow a tree.
        prob_fire: The probability that an inactive tree will catch on fire (e.g., by lightning).
        rng: The random number generator.

    Returns:
        The new state of the tree at position (x, y).
    """
    # Create a new grid to hold the updated states
    new_grid = grid.copy()

    # Rule 1: an empty node can be set to inactive with probability `prob_grow` (think of this as a tree growing).
    empty_nodes = grid == 0
    grow_mask = rng.uniform(size=grid.shape) < prob_grow
    new_grid[empty_nodes & grow_mask] = 1

    # Rule 2: an inactive node can become active with probability `prob_fire` (this is like a tree catching on fire).
    inactive_nodes = grid == 1
    fire_mask = rng.uniform(size=grid.shape) < prob_fire
    new_grid[inactive_nodes & fire_mask] = 2

    # Rule 3: an inactive node will always be set to active if any of its neighbouring nodes were active in the previous
    # iteration (this represents fire spreading from tree to tree).
    active_neighbors_mask = get_active_neighbors_mask(grid)
    new_grid[inactive_nodes & active_neighbors_mask] = 2

    # Rule 4: an active node will always become empty in the next iteration (this is like a tree burning down completely).
    new_grid[grid == 2] = 0

    return new_grid


# Re-initialize the random number generator.
rng = np.random.default_rng(123)

# Re-initialize the grid to all zeros.
grids = np.zeros((n_iter, grid_size, grid_size), dtype="int")

# Run the simulation with the newly-written simulation function
for step in trange(1, n_iter):
    grids[step] = apply_forest_fire_rule(grids[step - 1], prob_grow, prob_fire, rng)

Now, 1000 iterations take less than a second to simulate! Based on the iterations per second, we get over a 150x speedup. That's an excellent optimization! With this new implementation, we can now visualize the forest-fire simulation. As in the previous notebook, we'll not go into the details of the visualization code but instead, focus on the output. Below, we call our visualization function with the values we used previously. Unfortunately, despite our simulation code being well-optimized, the visualization code is not. This is because drawing 1000 plots and stitching them into an interactive visualization is computationally expensive. We'll make a similar compromise as we did with the ant agent simulation in the previous notebook. Rather than plotting every iteration, we'll plot every *tenth* iteration and plot 2000 iterations rather than 1000. This code cell takes approximately four minutes for the code to run. Additionally, because the animation has so many frames, we can't display it with the same tools as we did in the previous notebook. Instead, we have to save the animation to an MP4 file and load it back to view.

If you're interested, you can easily download the file once it finishes animating. Click on the folder icon on the left sidebar of the notebook. Then, you should see a file called `forest_fire.mp4`. Either right-click the file, or right-click the triple dots that appear when you hover over the file name and click "Download", to download it. Then, you can view it on a video player of your choice.

In [ ]:
# Increase the number of iterations to run the forest-fire simulation.
n_iter = 2000

# Only show every 10 iterations.
interval = 10

# Visualize the forest-fire model.
visualize_forest_fire(
    "forest_fire",
    grid_size,
    n_iter,
    prob_grow,
    prob_fire,
    interval,
    5000,
    200,
    seed=123,
)

Let's discuss the visualization. To assist in our analysis, we've also included the plots of the number of live trees (inactive nodes) vs. burning trees (active nodes) below the grid plot. Note that the lower plot uses two different scales on its y-axis. Recall as well that we only show every tenth iteration in the video. The start of the simulation is to be expected: trees start to pop up randomly in the grid. At iteration 222, we get our first lightning strike. However, the resulting fire lasts only two iterations, so we don't see it in the grid. We can only identify it by the small, red blip on the lower time series plot. In the end, only three trees burn down, and more are grown in these iterations, so the number of live trees continues to increase. We get a slightly larger fire at iteration 329; this one does appear on the grid very briefly in the lower-left corner. At iteration 440, a quick succession of lightning strikes burns much of the forest. This process of lightning burning down a lot of the forest, followed by the forest gradually renewing its trees, repeats several times in the 2000 iterations we've simulated. This results in the jagged plot you see of the live trees.

As per the rules of our simulation, the fires spread from one node to the next. The speed of the spread is affected by how many trees there are. The shape of the fire changes as it moves, depending on where the trees are located. The shape is also affected by how previous fires have changed the distribution of the trees in the forest. Previous fires can affect how many trees are burned in subsequent fires and how long those fires might last. Of course, if there are many trees, then fires will likely last for quite a long time, but that doesn't mean all fires will be the same. For example, consider the two fires at about iteration 1350 and iteration 1600. These fires start when there are about 3500 trees and last for about 100 iterations each. However, the second one burns a lot more trees than the first.

## Model Behaviour

So we've seen that our model produces a reasonably-believable forest fire simulation. As expected, the forest grows, and when lightning strikes, many trees are burned down in a large fire. However, how the fire burns and the patterns that emerge from the fire can vary greatly depending on where and when the fire started. How might we modify our model to get different results without changing the rules? For one, change the random number generator (RNG) seed. Changing this value will affect the stochastic rules; the exact pattern of tree growth and fire starts will be different. However, if visualized, the plots would be similar to what we see above. Of course, it would not be the same plot, but the same general jagged plot of live trees would be there.

Our model has two _free parameters_: parameters that we can change freely to affect how the model behaves. These parameters are the two probabilities affecting tree growth and tree ignition. In our example above, we used 0.001 (0.1%) for tree growth probability and 0.00001 (0.001%) for tree ignition probability. Since our plot was 100x100 nodes, on average, 100x100x0.001 = 10 new trees grow every iteration. Additionally, on average, 100x100x0.00001 = 0.1 trees spontaneously ignite every iteration, which is to say that we'd expect about one spontaneous ignition every ten iterations. How would the plots look if we modified these values? Below, we'll explore this a bit.

To begin, what if we _decreased_ the tree growth probability and _increased_ the tree ignition probability by an order of magnitude (divide and multiply by ten, respectively)?

In [ ]:
# Modify the growth and ignition probabilities.
prob_grow = 0.0001
prob_fire = 0.0001

# Visualize the forest-fire model.
visualize_forest_fire(
    "forest_fire_2",
    grid_size,
    n_iter,
    prob_grow,
    prob_fire,
    interval,
    2500,
    50,
    seed=123,
)

Again, please take note of the different y-axis scales in the lower plot. With these modified parameters such that the tree growth rate equals the tree ignition rate, the total number of trees increases very slowly. There are always a few small fires burning here and there, but they never get huge because there aren't enough trees close together. We rarely even see the fires in the grid plot since they are so short-lived. No significant drops in the number of live trees mean we don't see the jagged pattern we saw previously. We can also notice that the total number of live trees approaches a specific number rather than growing to infinity (or 100x100, in the case of our finite grid). This number is around 1500. This phenomenon is known as a _stationary state_ where the state of the forest is mainly stable. All simulations will reach such a state. In the previous example, despite the occasional large fires, the forest eventually recovered about as many trees as it had previously. Then, another fire would start, and this cycle would repeat. Suppose we continued the simulation for many more iterations. In that case, we'd see a similar pattern to the one in the plot. You can recognize the stationary state by noticing how the average number of trees (across some number of iterations) seems to have stabilized to a single value.

Next, let's increase the probability of tree growth by two orders of magnitude. This means that trees will grow at a rapid rate. We'll keep the probability of tree ignition the same at 0.0001. Recall that this corresponds to a rate of 100x100x0.001 = 1 tree ignitions every iteration.

In [ ]:
# Modify the growth and ignition probabilities.
prob_grow = 0.01
prob_fire = 0.0001

# Visualize the forest-fire model.
visualize_forest_fire(
    "forest_fire_3",
    grid_size,
    n_iter,
    prob_grow,
    prob_fire,
    interval,
    5000,
    500,
    seed=123,
)

As expected, when trees grow very fast, fires can spread quickly and keep going because new trees grow back almost as quickly as the fire spreads. Notice, however, how the forest still enters a stationary state despite its turbulent nature. Observe how the wavy patterns in both the time series of the live and burning trees seem to center on a particular value. It should also be noted that these and the previous set of parameters are unrealistic for real forests. Still, it's a helpful way to understand the possibilities of our forest-fire model. Let's now consider a more realistic situation where trees grow slowly, and fires rarely happen.

In [ ]:
# Modify the growth and ignition probabilities.
prob_grow = 0.001
prob_fire = 0.000001

# Visualize the forest-fire model.
visualize_forest_fire(
    "forest_fire_4",
    grid_size,
    n_iter,
    prob_grow,
    prob_fire,
    interval,
    8000,
    400,
    seed=123,
)

In this example, the forest has time to regrow trees before another fire starts. When a fire finally starts, it can spread through the entire forest because the trees are so close together. This leads to a cycle where the forest grows, a fire destroys almost everything, and then the forest starts growing again from almost nothing. When a fire starts in this situation, about half of the nodes in the grid are occupied by trees, which means almost every tree has a neighbour, and the fire can spread quickly. Some trees might survive a big fire because they're on the grid's edge and thus have fewer neighbours, making it more challenging for the fire to reach them.

## Criticality and Final Thoughts

In the first example of the forest-fire simulation, we saw a jagged pattern in the number of live trees in the grid. This corresponds with a behaviour in the forest where a fire starts at random and takes out many trees. In the last example, we made the difference between the model's two free parameters and noticed a similar pattern. However, since the rate at which fires start is lower, the time between large fires is longer. As a side-effect, since there are more trees the longer we let time pass between fires, the fires became much more significant, taking out more of the forest than in the first example. We don't observe this sort of behaviour when we set the probability equal (example two) or increase the growth rate (example three).

This leads us to consider a particular case, or regime, of parameters of `prob_grow` ≪ 1 and `prob_fire` ≪ `prob_grow`, where ≪ means "significantly less than". As Charbonneau states, in this regime, "the largest fires dominate the evolution of the (eco)system." We say that the system exhibits _self-organized criticality_. We are saying that the system naturally evolves to a point where one slight disturbance (such as a single tree getting set on fire) sets off a large-scale event (such as the entire forest being set alight). This happens regardless of the size of the grid.

Understanding this concept is critical to understanding how to manage and prevent wildfires. Why bother? Charbonneau cites three wildfires that have taken lives, devastated communities and ecosystems, and cost billions of dollars. And there certainly have been many more. At the time of writing this notebook, Canada is experiencing the worst wildfire season in North American history. Millions of acres of land have been burnt, and thousands of people have been evacuated. Air quality has deteriorated in many Canadian regions and in northern and midwestern United States. Smoke from the fires has even crossed the Atlantic.

We see in our simulations how quickly out-of-hand these large fires can get. Even if we were to try and put on small fires as quickly as possible, we need only one fire to reach the point of criticality for the forest fire to become uncontrollable. In fact, if we had let these small fires burn, they might have remained small. Since we put them out, that leaves those trees that would have been burnt to become fuel for the next uncontrollable forest fire and makes it easier for that fire to spread. As such, other strategies for managing wildfires must be employed (in addition to putting out small fires), such as having _planned ignitions_ to keep the density of trees manageable or the creation of _fire guards_, dirt roads clear of fuel for the fire and that act as barriers to prevent the spread of forest fires.

# References

- 2023 Canadian wildfires. (2023, July 3). In Wikipedia. https://en.wikipedia.org/wiki/2023_Canadian_wildfires
- Charbonneau, P. (2017). _Natural complexity: A modeling handbook_. Princeton University Press.
- Government of British Columbia. (n.d.). _Wildfire management strategies_. Retrieved July 3, 2023, https://www2.gov.bc.ca/gov/content/safety/wildfire-status/wildfire-response/management-strategies
- Petroff, M. A. (2021). _Accessible Color Sequences for Data Visualization_ (arXiv:2107.02270). arXiv. http://arxiv.org/abs/2107.02270